In [31]:
import os
import re
from time import time
import numpy as np
import pandas as pd
import cPickle as pickle

import openslide
from PIL import Image

from boto.s3.connection import S3Connection
from boto.s3.key import Key

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import metrics

In [7]:
conn = S3Connection()
bucket = conn.get_bucket('vegf')

In [8]:
slide_dfs = [
      #'df_70229.pkl'
      #,'df_70230.pkl'
      #,'df_70231.pkl'
      'df_70232.pkl' 
      #,'df_70233.pkl'
      #,'df_70234.pkl'
      #,'df_70235.pkl'
      #,'df_70236.pkl'
      #,'df_70237.pkl'
    ]

In [41]:
def bench_k_means(estimator, name, data):
    '''
    Source: http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html
    '''
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=1000)))

In [46]:
# get the tile metadata dataframe from S3 and load it with pickle
for slide_df in slide_dfs:
    
    #tile_df_filename = os.path.join(os.path.dirname(__file__), '../data/'+slide_df)
    tile_df_filename = '../data/'+slide_df
    if not os.path.exists(tile_df_filename):
        key = bucket.get_key(slide_df)
        key.get_contents_to_filename(tile_df_filename)
        
    with open(tile_df_filename, 'r') as f:
        df = pickle.load(f)
        
    X = df[range(2048)]
    
    for n in xrange(2,11):
        n_components = n
        n_clusters = n

        pca = PCA(n_components=n_components).fit(X)
        bench_k_means(KMeans(init=pca.components_, n_clusters=n_clusters, n_init=1), name="PCA-based n="+str(n_clusters), data=X)

PCA-based n=2   0.41s    555418   0.234
PCA-based n=3   0.85s    512880   0.151
PCA-based n=4   1.12s    489033   0.143
PCA-based n=5   0.79s    471311   0.132
PCA-based n=6   1.05s    457690   0.106
PCA-based n=7   0.81s    436214   0.133
PCA-based n=8   1.25s    428800   0.115
PCA-based n=9   1.60s    422485   0.125
PCA-based n=10   1.24s    421781   0.119


In [53]:
def cluster(df, n_clusters=2):
    X = df[range(2048)]
    pca = PCA(n_components=2).fit(X)
    kmeans = KMeans(init=pca.components_, n_clusters=2, n_init=1)
    kmeans.fit(X)
    df['cluster'] = kmeans.labels_
    return df

In [54]:
# get the tile metadata dataframe from S3 and load it with pickle
for slide_df in slide_dfs:
    
    #tile_df_filename = os.path.join(os.path.dirname(__file__), '../data/'+slide_df)
    tile_df_filename = '../data/'+slide_df
    if not os.path.exists(tile_df_filename):
        key = bucket.get_key(slide_df)
        key.get_contents_to_filename(tile_df_filename)
        
    with open(tile_df_filename, 'r') as f:
        df = pickle.load(f)
        
    df = cluster(df, n_clusters=2)

In [55]:
df.head()

,key,slide,width,height,x,y,0,1,2,3,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,cluster
633045,tiles/slide_70232_000167.tiff,70232,64,64,0.0,10688.0,0.490760,0.000000,0.150144,0.000000,...,0.002732,0.013804,0.151615,0.028847,0.020535,0.265548,0.060923,0.177500,0.585632,0
633046,tiles/slide_70232_000335.tiff,70232,64,64,64.0,10688.0,0.495601,0.000000,0.126711,0.000000,...,0.004705,0.025719,0.155366,0.000000,0.051949,0.208735,0.038782,0.233924,0.466447,0
633047,tiles/slide_70232_000498.tiff,70232,64,64,128.0,10368.0,0.007155,0.034675,0.018455,0.008901,...,0.073302,0.813239,0.865602,0.609654,0.113352,0.265793,0.375284,0.240248,0.529064,1
633048,tiles/slide_70232_000503.tiff,70232,64,64,128.0,10688.0,0.345169,0.000000,0.132178,0.000000,...,0.000000,0.064504,0.147059,0.006262,0.036226,0.255818,0.044558,0.187445,0.516848,0
633049,tiles/slide_70232_000671.tiff,70232,64,64,192.0,10688.0,0.467209,0.000000,0.143225,0.000000,...,0.023049,0.039450,0.107765,0.015839,0.035678,0.218351,0.056889,0.171210,0.489995,0


In [56]:
# Now, figure out how to overlay colors on the original slide.  Each tile gets a color overlay based on which cluster it belonged to.  